In [2]:
import numpy as np
import random

In [3]:
logging = False

def log(message):
    if logging:
        print(message)

In [4]:
import Organism

class World(Organism):
    def __init__(self, mx, my):
        self.maxX = mx
        self.maxY = my
        self.grid = np.array( [None] * (self.maxX * self.maxY)).reshape(self.maxX,self.maxY)
        self.lives = []
       
    def __repr__(self):
        return repr(self.grid)
    
    def biota(self):
        self.lives = []
        return list(filter(None,[n for sublist in self.grid for n in sublist]))
    
    def moveThing(self,coor, newcoor):
        self.x,self.y = coor
        newx,newy = newcoor
        self.grid[newy][newx] = self.grid[self.y][self.x]
        self.grid[self.y][self.x] = None
        
    def addThing(self , athing, coor):
        x,y = coor
        self.grid[y][x] = athing
    
    def delThing(self, x, y):
        self.grid[y][x] = None
        
    def lookAtLocation(self,coor):
        x,y = coor
        return self.grid[y][x]
    
    def emptyLocation(self,coor):
        x,y = coor
        return self.grid[y][x] == None
        
    def clear(self):
        self.grid = np.array( [None] * (self.maxX * self.maxY)).reshape(self.maxX,self.maxY)
    

In [5]:
class Organism():
    can_overcrowd = None
    breed_interval = None
    survive_without_food = None
    can_eat = None
    
    def __init__(self,name, world, coor):
        self.name = name
        self.world = world
        self.curcoor = coor
        self.xpos, self.ypos = coor
        self.world.addThing(self,coor)
        self.adjfish=[]
        self.adj=[]
        self.breedspan = self.breed_interval
        
    def __repr__(self):
        return repr(self.name)
    
    def location(self):
        return (self.xpos,self.ypos)
    
    def randomOffset(self):
        offsetList = [(-1,1),(0,1),(1,1),
                      (-1,0)      ,(1,0),
                      (-1,-1),(0,-1),(1,-1)]
        randomOffsetIndex = random.randrange(len(offsetList))
        randomOffset = offsetList[randomOffsetIndex]
        nextx = self.xpos + randomOffset[0]
        nexty = self.ypos + randomOffset[1]
        coor = (nextx % self.world.maxX ,nexty % self.world.maxY)
        return coor
            
    def adjacent(self):
        self.adj=[]
        offsetList = [(-1,1),(0,1),(1,1),
                      (-1,0)      ,(1,0),
                      (-1,-1),(0,-1),(1,-1)]
        
        for offset in offsetList:
            self.coor = ((self.xpos + offset[0])% self.world.maxX,(self.ypos + offset[1])% self.world.maxY)
            if(not self.world.emptyLocation(self.coor)):
                self.adj.append(self.world.lookAtLocation(self.coor)) 
    
    def reproduce(self):
        self.adjacent()
        self.a = list(map(lambda x: x.location(), self.adj))
        self.breed_interval = self.breed_interval-1
        if self.breed_interval == 0:
            self.breed_interval = self.breedspan
            self.rcoor = self.randomOffset()

            while self.rcoor in self.a:
                self.rcoor = self.randomOffset()
            self.birth(self.rcoor)
                               
    def overCrowding(self):
        self.adjacent()
        if self.can_overcrowd != None and len(list(filter(lambda x: x != self,self.adj)))>=self.can_overcrowd:
            self.world.delThing(self.xpos, self.ypos)
    
    def move(self):   
        pass
        
    def live(self):
        self.reproduce()
        self.overCrowding()

In [6]:
class Animal(Organism):
    def __init__(self,name, world, coor):
        Organism.__init__(self, self.name,world,coor)
        self.foodspan = self.survive_without_food 
    
    def eat(self):
        self.adjacent()
        if self.survive_without_food != None:
            if len(list(filter(lambda x: x == self.can_eat,self.adj))) >= 1:    
                randomOffsetIndex = random.randrange(len(list(filter(lambda x: x == self.can_eat,self.adj))))
                randomOffset = list(filter(lambda x: x == self.can_eat,self.adj))[randomOffsetIndex]
                x,y = randomOffset.location()
                self.world.delThing(x,y)
                self.survive_without_food = self.foodspan

            self.survive_without_food = self.survive_without_food-1
            if self.survive_without_food == 0:
                self.world.delThing(self.xpos, self.ypos)
                print("eat")
    
    def live(self):
        self.eat()
        self.reproduce()
        self.overCrowding()
        
    def move(self):
        self.adjacent()
        self.rcoor = self.randomOffset()
        self.a = list(map(lambda x: x.location(), self.adj))
        while self.rcoor in self.a:
            self.rcoor = self.randomOffset()

        self.world.moveThing(self.curcoor, self.rcoor)
            
        self.xpos, self.ypos = self.rcoor
        self.curcoor = self.rcoor
        print("move")

In [7]:
class Plant(Organism):
    can_overcrowd = 5
    breed_interval = 5
    def __init__(self, world, coor):
        self.name = '🌱'
        Organism.__init__(self, self.name,world,coor)
        
    def birth(self,coor):
        childThing = Plant(self.world,coor)
    

In [8]:
class Fish(Animal):
    can_overcrowd = 2
    survive_without_food = 8
    breed_interval = 8
    can_eat = Plant
    def __init__(self, world, coor):
        self.name = '🐟'
        Animal.__init__(self, self.name,world,coor)
        
    def birth(self,coor):
            childThing = Fish(self.world,coor)
    

In [9]:
class Bear(Animal):
    can_overcrowd = None
    survive_without_food = 10
    breed_interval = 12
    can_eat = Fish
    def __init__(self, world, coor):
        self.name = '🐻'
        Animal.__init__(self, self.name,world,coor)
        
    def birth(self,coor):
            childThing = Bear(self.world,coor)

        

In [10]:
def wbf(nrows, ncols, nbears, nfish):
    world = World(nrows, ncols)
    x = random.randrange(nrows)
    y = random.randrange(ncols)
    
    for n in range(nbears):
        while not world.emptyLocation((x,y)):
            x = random.randrange(nrows)
            y = random.randrange(ncols)
        Bear(world,(x,y))
        
    for n in range(nfish):
        while not world.emptyLocation((x,y)):
            x = random.randrange(nrows)
            y = random.randrange(ncols)
        Fish(world,(x,y))
    
    return world
        

In [11]:
def wbfp(nrows, ncols, nbears, nfish, nplants):
    world = World(nrows, ncols)
    x = random.randrange(nrows)
    y = random.randrange(ncols)
    
    for n in range(nbears):
        while not world.emptyLocation((x,y)):
            x = random.randrange(nrows)
            y = random.randrange(ncols)
        Bear(world,(x,y))
        
    
    for n in range(nfish):
        while not world.emptyLocation((x,y)):
            x = random.randrange(nrows)
            y = random.randrange(ncols)
        Fish(world,(x,y))
        
    for n in range(nplants):
        while not world.emptyLocation((x,y)):
            x = random.randrange(nrows)
            y = random.randrange(ncols)
        Plant(world,(x,y))
    
    return world
        

In [12]:
def step_system(world):
    for x in world.biota():
        x.live()
    for x in world.biota():
        x.move()

In [13]:
w = wbfp(10,10,3,5,20)
w

array([['🌱', None, None, None, None, '🐟', '🌱', None, None, '🌱'],
       [None, '🌱', None, None, '🌱', None, None, '🐟', None, None],
       ['🌱', None, None, '🌱', None, None, None, '🌱', '🌱', None],
       ['🌱', None, None, None, None, None, None, None, None, None],
       [None, None, None, '🐟', '🐟', None, None, None, None, None],
       [None, None, None, None, None, '🌱', None, None, '🌱', None],
       ['🌱', '🐻', None, None, None, None, None, None, None, '🐻'],
       [None, None, None, '🌱', None, '🐻', None, None, '🌱', None],
       [None, None, '🌱', None, None, '🌱', '🌱', '🌱', '🌱', None],
       [None, None, None, None, None, None, None, None, '🐟', None]], dtype=object)

In [16]:
w = World(10,10)
f = Bear(w,(5,5))
f2 = Plant(w,(6,5))
f3 = Plant(w,(5,6))
f4 = Plant(w,(4,5))
f5 = Plant(w,(5,4))
f6 =Plant(w,(6,4))
f7 = Plant(w,(4,6))
f8 =Plant(w,(6,6))

w

array([[None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, '🌱', '🌱', None, None, None],
       [None, None, None, None, '🌱', '🐻', '🌱', None, None, None],
       [None, None, None, None, '🌱', '🌱', '🌱', None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None]], dtype=object)

In [17]:
step_system(w)
w

move


array([[None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, '🐻', '🌱', '🌱', None, None, None],
       [None, None, None, None, '🌱', None, None, None, None, None],
       [None, None, None, None, '🌱', '🌱', '🌱', None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None]], dtype=object)

In [18]:
step_system(w)
w

move


array([[None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, '🐻', None, '🌱', '🌱', None, None, None],
       [None, None, None, None, '🌱', None, None, None, None, None],
       [None, None, None, None, '🌱', '🌱', '🌱', None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None]], dtype=object)

In [19]:
step_system(w)
w

move


array([[None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, '🐻', None, None, None, None, None],
       [None, None, None, None, None, '🌱', '🌱', None, None, None],
       [None, None, None, None, '🌱', None, None, None, None, None],
       [None, None, None, None, '🌱', '🌱', '🌱', None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None]], dtype=object)

In [20]:
step_system(w)
w

move


array([[None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, '🐻', None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, '🌱', '🌱', None, None, None],
       [None, None, None, None, '🌱', None, None, None, None, None],
       [None, None, None, None, '🌱', '🌱', '🌱', None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None]], dtype=object)

In [21]:
step_system(w)
w

move


array([[None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, '🐻', None, None, '🌱', None, None, None],
       [None, None, None, None, '🌱', '🌱', '🌱', None, None, None],
       [None, None, None, '🌱', '🌱', '🌱', None, None, None, None],
       [None, None, None, None, '🌱', '🌱', '🌱', None, None, None],
       [None, None, None, None, None, None, None, '🌱', None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None]], dtype=object)

In [22]:
step_system(w)
w

move


array([[None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, '🐻', None, None, None, None, None],
       [None, None, None, None, None, None, '🌱', None, None, None],
       [None, None, None, None, None, '🌱', '🌱', None, None, None],
       [None, None, None, '🌱', None, None, None, None, None, None],
       [None, None, None, None, '🌱', '🌱', '🌱', None, None, None],
       [None, None, None, None, None, None, None, '🌱', None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None]], dtype=object)

# AUTO-GRADER

In [30]:
# A new world has no objects
w1 = World(5,5)
assert len(w1.biota()) == 0

# After adding a fish there should be one object
f1 = Fish(w1, (2,2))
assert len(w1.biota()) == 1

# Test the location method
assert f1.location() == (2,2)

In [31]:
# Setting breed_interval to 1 should cause a fish to reproduce when live is called
w2 = World(5,5)
Fish.breed_interval = 1
f2 = Fish(w2, (2,2))
f2.live()
assert len(w2.biota()) == 2

# Reset the interval to original value for remaining tests
Fish.breed_interval = 12

In [32]:
# Make three fish, the one in the middle should die from overcrowding
w3 = World(5,5)
f3 = Fish(w3, (2,2))
Fish(w3, (1,1))
Fish(w3, (3,3))
f3.live()
assert len(w2.biota()) == 2

In [33]:
# When a fish moves it should be within one cell of its original location
w4 = World(5,5)
f4 = Fish(w4, (2,2))
f4.move()
r, c = f4.location()
assert (r,c) != (2,2)
assert abs(r-2) <= 1 and abs(c-2) <= 1

move


# AUTO-GRADER

In [34]:
# Test the Bear constructor and location method
w1 = World(5,5)
b1 = Bear(w1, (1,1))
assert len(w1.biota()) == 1
assert b1.location() == (1,1)

In [35]:
# Repeat the reproduction test for Bears
w2 = World(5,5)
Bear.breed_interval = 1
b2 = Bear(w2, (2,2))
b2.live()
assert len(w2.biota()) == 2
Bear.breed_interval = 8

eat


AssertionError: 

In [ ]:
# Make fish for the bear to eat, count the number of objects after eating
w3 = World(5,5)
b3 = Bear(w3, (2,2))
Fish(w3, (1,1))
Fish(w3, (3,3))
b3.live()
assert len(w2.biota()) == 2

In [36]:
# Setting the survival limit to 1 should cause a Bear to starve 
w4 = World(5,5)
Bear.survive_without_food = 1
b4 = Bear(w4, (2,2))
b4.live()
assert len(w4.biota()) == 0

eat


AssertionError: 

In [37]:
# Repeat the move test for bears
w5 = World(5,5)
b5 = Bear(w5, (2,2))
b5.move()
r, c = b5.location()
assert (r,c) != (2,2)
assert abs(r-2) <= 1 and abs(c-2) <= 1

move


In [38]:
w = wbf(10,10,3,12)

dct = { Bear: 0, Fish: 0 }
for x in w.biota():
    dct[x.__class__] += 1

assert dct[Bear] == 3
assert dct[Fish] == 12